# Data Exploration
## Virtual Data Collaboratory Undergraduate Data Science Workshop
## November 12, 2020

## Instructor: Ryan Womack

# Topics
- Introduction to Python and Jupyter Notebook environment
- Data handling and data exploration using descriptive statistics and data visualization
- Statistical analysis of a dataset using correlation and regression
- Introduction to the concept of training and testing data splits for machine learning and forecasting applications

## About Jupyter Notebooks and Python
Python is a general purpose computing language. Some of you may be familiar with it.  It is arguably the most popular language for data science applications due to the large number of statistical and computing modules that have been created for working with data.

The Jupyter Notebook is an interactive environment that allows Python code (and other programming languages) to be run alongside explanatory text (like this text), and to view results of code in the same document.  For this reason, it is particularly useful for learning and exploration.

## Executing Code
You can run the code cells in a Jupyter notebook by clicking in the cell and either clicking "Run" in the menu, or by selecting the keys "Control + Enter".  In Google Colab, you can also press the "Play" button icon.  The numbers that will appear to the left of the cell indicate the order in which commands are run.  It is important to run commands in the proper sequence.

## Importing Python modules
Using Python typically involves loading several specialized modules.  These modules contain many of the graphing and statistical tools we will use.  Run the following cell to import these modules and their specific features.  In this session we are focused on the data exploration process and not the Python language itself, but by seeing this code in action, you can get a sense of what your own Python programming might involve.

In [ ]:
# Run this cell, but please don't change it.

# These lines import Numpy and Pandas for data analysis, and Seaborn and Matplotlib for visualization

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import seaborn as sns
from statsmodels.api import OLS
from sklearn import linear_model

from google.colab import files

# "Magic" command below allows you to see plot in the notebook
%matplotlib inline

## Mount Google Drive

One additional piece of setup.  We need to first mount the Google Drive in Colab before we can read data from Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Working with Data
In most real-world data analysis situations, you must spend a significant amount of time to learn about your data and clean and prepare it for analysis.  We will work with a few datasets that have already been set up for you, so that you can focus on the concepts involved, instead of the details of data preparation.

We will work through one example togther, then you will have a choice of datasets to try out as an exercise.

# Life Expectancy

This dataset is an extract from the World Bank's Gender Statistics dataset, using 2018 data.  
https://datacatalog.worldbank.org/dataset/gender-statistics

We will look at *life expectancy* among females in various countries around the world, and try to understand some other variables that relate to it.

## Importing Data

We read data directly from its web location (the github site for this workshop).  

In [ ]:
life = pd.read_csv('https://github.com/ryandata/VDC_Data_Science_Workshop/raw/main/life.csv')

Print the data to see it and make sure that it is loaded correctly.

In [ ]:
print(life)

We use a few other commands to explore the data.  Columns lets us see the variable names. Shape lets us see the dimensions of the data. Info lets us see the variable formats.

In [ ]:
life.columns

In [ ]:
life.shape

In [ ]:
life.info()

Describe provides summary statistics of each variable

In [ ]:
life.describe()

Who has the highest life expectancy?
Let's sort the data and look at the top ranking countries.

In [ ]:
life.sort_values(by='Life_Expectancy', ascending=False).head()

## Visualize the relationship between two variables using a scatter plot

Plot each data point according to the value of two of the variables.  Observe the pattern that results.

Closely grouped and steeply sloped lines will indicate a strong relationship between variables.  Note that the relationship may not always be linear.  Learning to work with more complex models that capture these nuances of the relationships is a key part of data science. 

Note: We are using the *seaborn* package for visualization in Python.  Along with *matplotlib*, it is one of the most commonly used visualization tools for Python.

First, we compare Life Expectancy to Mortality Rate.  What can we say about this relationship?

In [ ]:
x_value = life['Mortality_Rate']
y_value = life['Life_Expectancy']

with sns.axes_style('ticks'):
    sns.scatterplot(x=x_value, y=y_value)

plt.show()

We can also compare all variables to each other using a pair plot.

In [ ]:
sns.pairplot(life)

We can also look at how data is distributed--for example, Life Expectancy.  One way is via a histogram.

In [ ]:
sns.displot(y_value)


Another is by estimating the distribution of the data (Kernel Density Estimation)

In [ ]:
sns.displot(y_value, kind="kde")



Or we can combine both on the same plot.

In [ ]:
sns.displot(y_value, kde="True")

Our final visualization returns us to the scatterplot, but with a **regression line** through the data.  This line is the best linear fit to the data, and can be viewed as estimating the relationship between x and y, if that relationship were reduced to a linear equation.  The process of computing this estimate is called **regression**.

In [ ]:
x_value = life['Mortality_Rate']
y_value = life['Life_Expectancy']

# Plotting regression, with figure-specific style change to white/grid-less background
with sns.axes_style('ticks'):
    sns.regplot(x=x_value, y=y_value, scatter_kws={'s':10})

plt.show()

# Correlation and Regression

## Correlation
**Correlation** is a measure of the strength of the relationship between two variables, ranging from -1 (strongest negative relationship) to 0 (no relationship) to 1 (strongest positive relationship). The relationship may be causal or simply an association, but if a correlation is present, knowing something about one variable means we can make an educated guess, or prediction, about the second variable.
We can test the correlation between two variables:

In [ ]:
life.corr()

In [ ]:
Or we can look at all of the correlations in a dataset.

In [ ]:
life.corr()

Here "Probability of Survival to Age 5" has the highest correlation (in absolute value) to Life Expectancy, so it might be one of the first variables to consider when building a regression model to explain the relationship.

In [ ]:
We can also visualize the correlation matrix.

In [ ]:
corr_mat = life.corr().stack().reset_index(name="correlation")
sns.relplot(
    data=corr_mat,
    x="level_0", y="level_1", hue="correlation", size="correlation",
    palette="vlag", hue_norm=(-1, 1), edgecolor=".7",
    height=10, sizes=(50, 250)
)


## Regression

Our final step will be to actually compute the regression equation.  Here we build a model with some of the likely variables, and see how much explanatory power it has.  Our equation will be of the form
*y* = *c* + a1*x1* + a2*x2* + ... an*xn*
where *y* is our *response* variable (or dependent variable), which we are trying to explain. The *x* variables represent the *explanatory* variables (or independent variables) that we believe affect *y*.  The a's are coefficients on the *x* variables. Finally, *c* is a constant that will be computed by the regression process.
The key is to choose the *x* variables wisely. We will start with the one that has the highest correlation to *y*, and then move down the list.
In the case of Life Expectancy, we found that Probablity of Survival to age 5 was most closely related, so we build the regression as follows.

Note that we are using the linear regression tools from *scikit-learn*.  Scikit-learn (https://scikit-learn.org/) is one of the most powerful and popular tools for machine learning in Python.

Specify our Y and X for the model

In [ ]:
y = life.iloc[:, 'Life_Expectancy'].values
print(Y)

In [ ]:
X=life[['Prob_Survival_to_5']]
print(X)

The next step is necessary to convert the data into a form that the regression algorithm can process.

Create linear regression object

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:
Fit the model

In [ ]:
regr.fit(X, Y)

In [ ]:
Plot the results

In [ ]:
plt.plot(X, regr.predict(X), color='red',linewidth=3)
plt.plot(X,Y,'o')

We can also get some very detailed statistics on the regression fit.  For now, just look at two things in the output below.  
*R-squared* is one measure of how good the fit it.  It ranges from 0 to 1, with 1 meaning that the equation completely explains all of the variance in the data (best fit).  There is a bit more to it than that, but that is a good enough approximation for now.
The *coef* is the coefficient on our explanatory variable.  It this case, it means we can take the Probability of Survival to Age 5 and multiply it by 77 to get an estimate of Life Expectancy that will be reasonably accurate.
This gives you a sense of how regression can work as a predictor. We'll return to that topic later.

In [ ]:
OLS(Y,X).fit().summary()

We can also consider multiple regression, with more than one explanatory variable, along with many other variations.  In the interest of time, however, we'll just be using one variable models in this workshop.

# Exercises/Case Studies
What follows are 4 case studies that you will work through as individual exercises.  Choose ONE that interests you, and explore the data using our tools for data exploration, visualization, and regression.
The cases are:
    - Cardiovascular Disease (Science)
    - World Development (Social Science)
    - World Economy (Business/Economics)
    - Environment (Another Science)
    
In each case, there are only a few "moving parts" - cells where you will edit the code to select variables and parameters.  Look for the **EDIT HERE** statement.  Be careful to run cells in order.

## Case Study 1: Cardiovascular Disease

The original data has records of 70,000 patients.  See
https://www.kaggle.com/sulianova/cardiovascular-disease-dataset for details.  The dataset presented for this class is a sample of 1000 patients (for rapid analysis), and has been cleaned of outliers.

The variable "cardio" is an indicator of whether or not cardiovascular disease is present.  For now, let's try to predict what is linked to high blood pressure (ap_hi in the dataset).

In [ ]:
cardio = pd.read_csv('https://github.com/ryandata/VDC_Data_Science_Workshop/raw/main/cardio.csv')

In [ ]:
cardio.columns

In [ ]:
cardio.shape

In [ ]:
cardio.info()

In [ ]:
cardio.describe()

Some variables are categorical - for example, the cholesterol level is given a rating from 1 to 3 (good to worst). For a zero-one variable like smoking, a zero typically means "no" (in this case non-smoking) or "yes" (smoking).

In [ ]:
cardio.sort_values(by='weight', ascending=False).head()

In [ ]:
sns.pairplot(cardio)

Here the pairplot is a bit hairy.  There are many variables, and the categorical variables are hard to interpret with this kind of visualization.  We can move on to examining individual combinations.

In [ ]:
x_value = cardio['weight']
y_value = cardio['ap_hi']

# Plotting regression, with figure-specific style change to white/grid-less background
with sns.axes_style('ticks'):
    sns.regplot(x=x_value, y=y_value, scatter_kws={'s':10})

plt.show()

In [ ]:
sns.displot(y_value)


In [ ]:
sns.displot(y_value, kde="True")